In [1]:
import os
# 한글 지원
os.putenv('NLS_LANG', '.UTF8')

In [2]:
def ipchecker():    
    import urllib 
    import re
    
    pattern = re.compile(b'\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}')
    dyndns = urllib.request.urlopen('http://checkip.dyndns.org').read()
    
    wan = re.search(pattern, dyndns).group().decode('utf-8')
    
    return wan

In [3]:
# base db_connect param

local_ip = '200.1.220.251:1521/xe' # 원내에서 접속할 db_ip
outside_ip = '61.35.152.103:1521/xe' # 원밖에서 접속할 db_ip
localhost = 'localhost:1521/xe'

db_id = 'begopa' 
db_pwd = 'begopa1'

academy_ip = '61.35.152.66'
db_ip = local_ip if academy_ip == ipchecker() else outside_ip

# db_ip = localhost # 집에서 본인의 개인 컴퓨터dptj  오라클에 접속할때는

In [4]:
import pickle
# load object ::
# :: file_nm
# :: columns and data_type
with open("./pickle/col2dict.pickle","rb") as fr:
    col2dict = pickle.load(fr)
with open("./pickle/table_nm_set.pickle","rb") as fr:
    table_nm_set = pickle.load(fr)

In [5]:
path = "./sg-data/"
desc_path = path + 'desc/'
col_path = path + 'col/'
origin_path = path + 'origin/'

file_list = os.listdir(origin_path)

In [6]:
# _, _, _, table_col_set_ = extract_table_columns_set(file_list[0]) # _, :: 임시로 쓰고 버리는 값.

In [7]:
# 자치구별 상권 변화지표 db 에서 한글 컬럼으로 가져오기
# oracle_to_df('자치구별 상권변화지표')
def load_to_df(kr_tbl_nm):
    for key, value in table_nm_set.items():
        if kr_tbl_nm == value:
            table_nm = key
            columns = [ arr[0] for arr in col2dict[key] ]
           
    import cx_Oracle
    import pandas as pd
    
    db = cx_Oracle.connect(db_id, db_pwd, db_ip) 
        
    sql = 'select * from ' + table_nm

    df = pd.read_sql(sql=sql, con = db) 
    df.columns = columns
    db.close
    
    return df


In [8]:
import re
import os

file_list = os.listdir(origin_path)
for file in file_list:
    kr_tbl_nm = re.findall('(\d\d).*\((.*)\)',file)
    print(kr_tbl_nm[0][0], kr_tbl_nm[0][1])

00 상권-영역
01 상권-추정유동인구
02 상권-집객시설
03 상권배후지-추정유동인구
04 상권배후지-집객시설
05 상권배후지-상주인구
06 상권_상주인구
07 상권배후지-아파트
08 상권배후지-점포
09 상권-상권변화지표
10 행정동별 상권변화지표
11 상권-추정매출
12 상권-직장인구
13 상권배후지-소득소비
14 상권배후지-직장인구
15 자치구별 상권변화지표
16 상권-아파트
18 상권-점포
19 상권배후지-추정매출


In [9]:
import pandas as pd

df = load_to_df('상권-추정유동인구')
df1 = df.copy()
# print(df1.head())
df1[df1['기준_년_분기_코드']=='2019_1']
df1[df1['기준_년_분기_코드']=='2019_2']


# df1.filter(like='여성').filter(like='토요일')
df1['여성_토요일_유동인구수'] = df1.filter(like='여성').filter(like='토요일').sum(axis=1)
df1['여성_토요일_유동인구수']
del_columns = df1.filter(like='여성').filter(like='토요일').filter(like='시간대').columns
df1.drop(del_columns, axis=1, inplace=True)
df1.filter(like='여성').filter(like='토요일')

delete_dict = {'여성' : ['일요일','월요일','화요일','수요일','목요일','금요일','토요일'],
               '남성' : ['일요일','월요일','화요일','수요일','목요일','금요일','토요일'],
              }
delete_word = '시간대'




# len(df1.columns), len(df1.filter(like='유동인구_수').columns)
# df1.iloc[:,:5].head(2), df1.filter(like='유동인구_수').iloc[:,:5].head(2)
# df1['여성_토요일_유동인구수']
# df1.filter(items=['여성', '토요일'])

,기준_년_분기_코드,상권_코드,총_유동인구_수,남성_유동인구_수,여성_유동인구_수,연령대_10_유동인구_수,연령대_20_유동인구_수,연령대_30_유동인구_수,연령대_40_유동인구_수,연령대_50_유동인구_수,...,여성연령대_60_이상_금요일시간대_4_유동인구_수,여성연령대_60_이상_금요일시간대_5_유동인구_수,여성연령대_60_이상_금요일시간대_6_유동인구_수,여성연령대_60_이상_일요일시간대_1_유동인구_수,여성연령대_60_이상_일요일시간대_2_유동인구_수,여성연령대_60_이상_일요일시간대_3_유동인구_수,여성연령대_60_이상_일요일시간대_4_유동인구_수,여성연령대_60_이상_일요일시간대_5_유동인구_수,여성연령대_60_이상_일요일시간대_6_유동인구_수,여성_토요일_유동인구수
0,2019_1,1000001,118266,61268,56997,10770,23173,24546,23024,20906,...,251,178,27,31,281,410,219,129,25,9637
1,2019_1,1000002,246619,147358,99261,11348,46047,44022,39844,49312,...,681,377,103,170,493,957,1175,439,85,16279
2,2019_1,1000003,144480,80073,64407,4000,39807,30677,24943,24117,...,283,253,69,63,92,152,168,149,38,10198
3,2019_1,1000012,166390,70339,96051,20594,41192,31759,29995,25683,...,264,253,113,182,191,254,244,190,76,12770
4,2019_1,1000013,65518,33437,32080,1506,22078,14341,10796,9384,...,119,121,18,9,24,55,62,26,6,5803
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2985,2019_2,1001325,109383,60736,48646,833,10425,11600,13788,26118,...,927,704,117,52,202,484,709,654,97,6862
2986,2019_2,1001379,112258,51637,60622,7751,23010,17053,18041,22142,...,562,417,115,108,199,297,247,250,75,8165
2987,2019_2,1001380,134058,70806,63253,9881,24125,17515,24234,27932,...,312,321,198,530,556,612,411,328,151,9254
2988,2019_2,1001381,39190,18328,20862,3441,7451,7122,7097,6629,...,124,119,52,79,91,120,127,99,43,2960


In [10]:
def delete_age_by_time(df1, delete_dict, delete_word):
    # df1.filter(like='여성').filter(like='토요일')
    for key, values in delete_dict.items():
        for value in values:
            df1[key+'_'+value+'_유동인구수'] = df1.filter(like=key).filter(like=value).sum(axis=1)
            del_columns = df1.filter(like=key).filter(like=value).filter(like=delete_word).columns
            df1.drop(del_columns, axis=1, inplace=True)
    
    return df1    

In [11]:
df1 = delete_age_by_time(df1, delete_dict, delete_word)
df1.columns

Index(['기준_년_분기_코드', '상권_코드', '총_유동인구_수', '남성_유동인구_수', '여성_유동인구_수',
       '연령대_10_유동인구_수', '연령대_20_유동인구_수', '연령대_30_유동인구_수', '연령대_40_유동인구_수',
       '연령대_50_유동인구_수', '연령대_60_이상_유동인구_수', '시간대_1_유동인구_수', '시간대_2_유동인구_수',
       '시간대_3_유동인구_수', '시간대_4_유동인구_수', '시간대_5_유동인구_수', '시간대_6_유동인구_수',
       '월요일_유동인구_수', '화요일_유동인구_수', '수요일_유동인구_수', '목요일_유동인구_수', '금요일_유동인구_수',
       '토요일_유동인구_수', '일요일_유동인구_수', '여성_토요일_유동인구수', '여성_일요일_유동인구수',
       '여성_월요일_유동인구수', '여성_화요일_유동인구수', '여성_수요일_유동인구수', '여성_목요일_유동인구수',
       '여성_금요일_유동인구수', '남성_일요일_유동인구수', '남성_월요일_유동인구수', '남성_화요일_유동인구수',
       '남성_수요일_유동인구수', '남성_목요일_유동인구수', '남성_금요일_유동인구수', '남성_토요일_유동인구수'],
      dtype='object')

In [18]:
df1['총_시간_유동인구_수'] = df1.filter(like='시간대_').filter(like='_유동인구_수').sum(axis=1)
df1['총_연령_유동인구_수'] = df1.filter(like='연령대_').filter(like='0_유동인구_수').sum(axis=1)

del_columns = df1.filter(like='시간대_').filter(like='_유동인구_수').columns
# df1.drop(del_columns, axis=1, inplace=True)

del_columns = df1.filter(like='연령대_').filter(like='_유동인구_수').columns
# df1.drop(del_columns, axis=1, inplace=True)
df1['총_성별_유동인구_수'] = df1.filter(like='성_유').sum(axis=1)

In [19]:
df1.filter(like='총_')

,총_유동인구_수,총_시간_유동인구_수,총_연령_유동인구_수,총_성별_유동인구_수
0,118266,118267,102419,118265
1,246619,246621,190573,246619
2,144480,144479,123544,144480
3,166390,166387,149223,166390
4,65518,65518,58105,65517
...,...,...,...,...
2985,109383,109381,62764,109382
2986,112258,112258,87997,112259
2987,134058,134058,103687,134059
2988,39190,39190,31740,39190
